# Design of an Object Oriented Data Model

## 

1. **Field Type Classification** – Identify each field as primitive, optional, collection, or nested.
2. **Per-Field Effort Weights** – Assign LOC and complexity contributions for each category.
3. **Estimator Refactor** – Generalize estimators to accept a list of field types.
4. **Effort Aggregation** – Accumulate weighted LOC and complexity.

### Field Type Classification


In [99]:
from typing import Callable, Dict

# Individual estimators

def estimate_to_string_effort(num_fields: int) -> dict:
    loc = num_fields
    complexity = 1
    return {"loc": loc, "complexity": complexity}

def estimate_equals_effort(num_fields: int) -> dict:
    loc = num_fields + 1
    complexity = 2
    return {"loc": loc, "complexity": complexity}

def estimate_hash_code_effort(num_fields: int) -> dict:
    loc = num_fields + 1
    complexity = 2
    return {"loc": loc, "complexity": complexity}

def estimate_copy_with_effort(num_fields: int) -> dict:
    loc = num_fields + 2
    complexity = 3
    return {"loc": loc, "complexity": complexity}

def estimate_to_json_effort(num_fields: int) -> dict:
    loc = num_fields + 2
    complexity = 2
    return {"loc": loc, "complexity": complexity}

def estimate_from_json_effort(num_fields: int) -> dict:
    loc = num_fields + 2
    complexity = 3
    return {"loc": loc, "complexity": complexity}

def estimate_constructor_effort(num_fields: int) -> dict:
    loc = num_fields + 1
    complexity = 1
    return {"loc": loc, "complexity": complexity}


# Registry of estimators
effort_estimators: Dict[str, Callable[[int], dict]] = {
    "toString": estimate_to_string_effort,
    "equals": estimate_equals_effort,
    "hashCode": estimate_hash_code_effort,
    "copyWith": estimate_copy_with_effort,
    "toJson": estimate_to_json_effort,
    "fromJson": estimate_from_json_effort,
    "constructor": estimate_constructor_effort,
}



In [100]:
def classify_field_type(type_str: str) -> str:
    """
    Classify a type string into one of: primitive, optional, collection, nested.
    """
    primitive_types = {"int", "double", "bool", "String", "number"}
    
    if type_str.endswith("?"):
        return "optional"
    elif type_str in primitive_types:
        return "primitive"
    elif type_str.startswith("List") or type_str.startswith("Map"):
        return "collection"
    else:
        return "nested"


### Complexity Weights

In [101]:
FIELD_COMPLEXITY_WEIGHTS = {
    "primitive": {"loc": 1, "complexity": 1},
    "optional": {"loc": 1, "complexity": 2},
    "collection": {"loc": 2, "complexity": 3},
    "nested": {"loc": 2, "complexity": 4},
}


### Generalized Estimator Utility

We define a helper to compute weighted totals across field types:


In [102]:
from typing import List

def compute_weighted_effort(field_types: List[str], base_loc=0, base_complexity=0, scaling=1.0) -> dict:
    """
    Aggregate LOC and complexity for a list of field types, with optional base values.
    """
    total_loc = base_loc
    total_complexity = base_complexity

    for t in field_types:
        category = classify_field_type(t)
        weights = FIELD_COMPLEXITY_WEIGHTS[category]
        total_loc += scaling * weights["loc"]
        total_complexity += scaling * weights["complexity"]

    return {
        "loc": int(round(total_loc)),
        "complexity": int(round(total_complexity)),
    }


### Estimators



In [103]:

def estimate_to_string_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=0, base_complexity=1)

def estimate_equals_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=1, base_complexity=1)

def estimate_copy_with_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=2, base_complexity=2)

def estimate_validation_class_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=3, base_complexity=2, scaling=2)

def estimate_transformer_class_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, scaling=2)



In [104]:
### Registry

In [105]:
typed_effort_estimators = {
    "toString": estimate_to_string_effort_typed,
    "equals": estimate_equals_effort_typed,
    "copyWith": estimate_copy_with_effort_typed,
    "validationClass": estimate_validation_class_effort_typed,
    "transformerClass": estimate_transformer_class_effort_typed,
    # more as needed...
}


### Effort Summary Function

In [106]:
def estimate_total_effort_typed(field_types: List[str], selected_functions=None) -> dict:
    if selected_functions is None:
        selected_functions = list(typed_effort_estimators.keys())

    breakdown = {}
    total_loc = 0
    total_complexity = 0

    for name in selected_functions:
        estimator = typed_effort_estimators.get(name)
        if estimator:
            result = estimator(field_types)
            breakdown[name] = result
            total_loc += result["loc"]
            total_complexity += result["complexity"]

    return {
        "total_loc": total_loc,
        "total_complexity": total_complexity,
        "breakdown": breakdown
    }



### Example Usage

In [107]:

field_types = ["int", "String", "List<int>", "Map<String, dynamic>", "User", "bool?"]
report = estimate_total_effort_typed(field_types)
report


{'total_loc': 69,
 'total_complexity': 104,
 'breakdown': {'toString': {'loc': 9, 'complexity': 15},
  'equals': {'loc': 10, 'complexity': 15},
  'copyWith': {'loc': 11, 'complexity': 16},
  'validationClass': {'loc': 21, 'complexity': 30},
  'transformerClass': {'loc': 18, 'complexity': 28}}}

In [108]:
from typing import Union

def estimate_total_effort(
    fields: Union[int, List[str]],
    selected_functions: List[str] = None
) -> dict:
    """
    Unified interface for estimating total implementation effort.

    Parameters:
    - fields: Either an integer (number of fields) or a list of field type strings.
    - selected_functions: Optional list of utility names to include.

    Returns:
    - Dictionary with total LOC, total complexity, and per-function breakdown.
    """
    if isinstance(fields, int):
        # Use untyped estimators
        estimators = effort_estimators
        get_estimate = lambda fn: estimators[fn](fields)
    elif isinstance(fields, list):
        # Use typed estimators
        estimators = typed_effort_estimators
        get_estimate = lambda fn: estimators[fn](fields)
    else:
        raise ValueError("`fields` must be either an integer or a list of type strings.")

    if selected_functions is None:
        selected_functions = list(estimators.keys())

    breakdown = {}
    total_loc = 0
    total_complexity = 0

    for fn in selected_functions:
        if fn in estimators:
            result = get_estimate(fn)
            breakdown[fn] = result
            total_loc += result["loc"]
            total_complexity += result["complexity"]

    return {
        "total_loc": total_loc,
        "total_complexity": total_complexity,
        "breakdown": breakdown
    }


Untyped: using just the field count


In [109]:
estimate_total_effort(fields=5)

{'total_loc': 44,
 'total_complexity': 14,
 'breakdown': {'toString': {'loc': 5, 'complexity': 1},
  'equals': {'loc': 6, 'complexity': 2},
  'hashCode': {'loc': 6, 'complexity': 2},
  'copyWith': {'loc': 7, 'complexity': 3},
  'toJson': {'loc': 7, 'complexity': 2},
  'fromJson': {'loc': 7, 'complexity': 3},
  'constructor': {'loc': 6, 'complexity': 1}}}

Typed: using list of field types


In [110]:
field_types = ["int", "String", "List<int>", "User", "bool?"]
estimate_total_effort(fields=field_types)

{'total_loc': 55,
 'total_complexity': 83,
 'breakdown': {'toString': {'loc': 7, 'complexity': 12},
  'equals': {'loc': 8, 'complexity': 12},
  'copyWith': {'loc': 9, 'complexity': 13},
  'validationClass': {'loc': 17, 'complexity': 24},
  'transformerClass': {'loc': 14, 'complexity': 22}}}

### A - Flat Model

All fields live in a single "mega" concept (class) — say, 100 primitive fields

In [111]:
def build_flat_model(num_fields: int) -> dict:
    """
    Construct a flat model where all fields are primitive.
    
    Returns:
        Dictionary representing a single concept with all fields.
    """
    field_types = ["int"] * num_fields
    return {
        "FlatConcept": field_types
    }


### B - Modular Model

Those same fields are distributed across child concepts, and the parent concept holds references to those. 

In [112]:
def build_modular_model(total_fields: int, fields_per_child: int) -> dict:
    """
    Constructs a modular model:
    - Divides fields into child concepts.
    - A parent concept holds references to the child concepts.

    Returns:
        Dictionary mapping concept names to their list of field types.
    """
    if total_fields % fields_per_child != 0:
        raise ValueError("total_fields must be divisible by fields_per_child.")

    num_children = total_fields // fields_per_child
    model = {}

    # Create child concepts
    for i in range(num_children):
        concept_name = f"ChildConcept{i + 1}"
        model[concept_name] = ["int"] * fields_per_child

    # Create parent concept with references to all child concepts
    parent_fields = list(model.keys())  # Treat each child as a field (reference)
    model["ParentConcept"] = parent_fields  # These are nested references

    return model

### Estimation per Model

In [113]:
def estimate_model_effort(model: dict, selected_functions: List[str] = None) -> dict:
    """
    Estimate total effort for a model with optional function filtering.
    """
    total_loc = 0
    total_complexity = 0
    concept_breakdown = {}

    for concept_name, field_types in model.items():
        result = estimate_total_effort(fields=field_types, selected_functions=selected_functions)
        concept_breakdown[concept_name] = result
        total_loc += result["total_loc"]
        total_complexity += result["total_complexity"]

    return {
        "total_loc": total_loc,
        "total_complexity": total_complexity,
        "concept_breakdown": concept_breakdown
    }


# Comparison Framework

In [114]:
def compare_flat_vs_modular(
    total_fields: int,
    fields_per_child: int,
    selected_functions: List[str] = None
) -> dict:
    """
    Run A/B test between flat and modular models using selected utilities.
    """
    flat_model = build_flat_model(total_fields)
    modular_model = build_modular_model(total_fields, fields_per_child)

    flat_effort = estimate_model_effort(flat_model, selected_functions)
    modular_effort = estimate_model_effort(modular_model, selected_functions)

    return {
        "flat_model": flat_effort,
        "modular_model": modular_effort,
        "delta": {
            "loc": modular_effort["total_loc"] - flat_effort["total_loc"],
            "complexity": modular_effort["total_complexity"] - flat_effort["total_complexity"]
        }
    }


Example:

In [ ]:
result = compare_flat_vs_modular(100, 10, selected_functions=["toString", "equals"])
result

{'flat_model': {'total_loc': 706,
  'total_complexity': 706,
  'concept_breakdown': {'FlatConcept': {'total_loc': 706,
    'total_complexity': 706,
    'breakdown': {'toString': {'loc': 100, 'complexity': 101},
     'equals': {'loc': 101, 'complexity': 101},
     'copyWith': {'loc': 102, 'complexity': 102},
     'validationClass': {'loc': 203, 'complexity': 202},
     'transformerClass': {'loc': 200, 'complexity': 200}}}}},
 'modular_model': {'total_loc': 906,
  'total_complexity': 1046,
  'concept_breakdown': {'ChildConcept1': {'total_loc': 76,
    'total_complexity': 76,
    'breakdown': {'toString': {'loc': 10, 'complexity': 11},
     'equals': {'loc': 11, 'complexity': 11},
     'copyWith': {'loc': 12, 'complexity': 12},
     'validationClass': {'loc': 23, 'complexity': 22},
     'transformerClass': {'loc': 20, 'complexity': 20}}},
   'ChildConcept2': {'total_loc': 76,
    'total_complexity': 76,
    'breakdown': {'toString': {'loc': 10, 'complexity': 11},
     'equals': {'loc': 11